In [11]:
import lancedb
import pandas as pd
import pyarrow as pa

In [4]:
uri = "data/sample-lancedb"
db = lancedb.connect(uri)

In [7]:
db.uri

'/home/mohitm/projects/obsidian-rag-chat/data/sample-lancedb'

In [8]:
data = [
    {"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
    {"vector": [5.9, 26.5], "item": "bar", "price": 20.0},
]

# Synchronous client
tbl = db.create_table("my_table", data=data)

In [9]:
tbl

LanceTable(connection=LanceDBConnection(/home/mohitm/projects/obsidian-rag-chat/data/sample-lancedb), name="my_table")

In [15]:
#creating empty table


schema = pa.schema([pa.field("vector", pa.list_(pa.float32(),list_size=2))])

#sync client
empty_tbl = db.create_table("empty_table", schema=schema)

In [18]:
#open an existing table

tbl = db.open_table('my_table')


In [19]:
data = [
    {"vector": [3.11, 3.1], "item": "foo3", "price": 12.0},
    {"vector": [59.0, 2.5], "item": "bar2", "price": 210.0},
]

tbl.add(data)


In [24]:
tbl.search([40, 2.5]).limit(2).to_pandas()

,vector,item,price,_distance
0,"[59.0, 2.5]",bar2,210.0,361.000000
1,"[3.11, 3.1]",foo3,12.0,1361.232056


In [25]:
#embedding API

from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry


In [27]:
db = lancedb.connect("/tmp/db")
func = get_registry().get("openai").create(name = 'text-embedding-3-small')

class Words(LanceModel):
    text: str=  func.SourceField()
    vector : Vector(func.ndims()) = func.VectorField()
    


In [30]:
table = db.create_table('words', schema=Words, mode='overwrite')
table.add([{"text": "hello world"}, {"text": "goodbye world"}])

query = "greetings"
actual = table.search(query).limit(1).to_pydantic(Words)[0]
print(actual.text)

 Retrying in 3.2666150552770805 seconds (retry 1 of 7) 

 Retrying in 7.202553897720034 seconds (retry 2 of 7) 



KeyboardInterrupt: 